In [ ]:
from conditional_substitution import conditional_substitution
from identities_common import *

R.<u, x, q, v> = ZZ[]
Z = polygen(R, 'Z')
K = R.fraction_field()
a = u*(1-u*x)/(1+u)/(1-u^2*x)
b = u^2*x * (1-u)^2 / (1+u)^2 / (1-u^2*x)^2
Delta = (1-u)/(1+u)
delta_square = (1-x)/(1-u^2*x)

In [ ]:
R1.<delta> = K.extension(Z^2 - delta_square, 'delta')

In [ ]:
branch_rules = {
    (SR((u^2-2*u+1)/(SR(u)+1)^2))^(j/2): ((1-u)/(1+u))^j for j in srange(-21, 22, 2)
}

In [ ]:
assert Delta^2 == (1-2*a)^2-4*b

## All Vertices

In [ ]:
c0 = Delta^2
c1 = (2*a-4*a^2 + 4*b)
c2 = 4*a^2-4*b
assert (1-2*a*(1+q))^2 - 4*b*(1+q)^2 == (c0 -2*q*c1+c2*q^2)

In [ ]:
c2_Delta_root = 2*u/(1-u)*delta

In [ ]:
assert c2_Delta_root^2 == c2/Delta^2

In [ ]:
alpha_minus_beta = c2_Delta_root

In [ ]:
alpha_plus_beta = (c1/Delta^2).factor()

In [ ]:
alpha = (alpha_plus_beta + alpha_minus_beta)/2

In [ ]:
beta = (alpha_plus_beta - alpha_minus_beta)/2

In [ ]:
alpha_S = SR(alpha.lift()).subs(delta=sqrt(SR(delta_square)))
beta_S =  SR(beta.lift()).subs(delta=sqrt(SR(delta_square)))

In [ ]:
identity_all = -L(SR(a)*(1+q), b*(1+q)^2)  + Delta*(1/Delta-1+q*(alpha_S-beta_S))/2 + Delta*T(alpha_S*q, beta_S*q)

In [ ]:
def custom_sqrt(d):
    radicand = d[SR.wild(0)].factor()
    new_radicand = 1
    extracted = 1
    for operand in radicand.operands():
        if operand == 1/(u^2*x-1)^2:
            extracted *= 1/(1-u^2*x)
        elif operand == 1/(u^2*x-1)^4:
            extracted *= 1/(1-u^2*x)^2
        elif operand == 1/(u-1)^2:
            extracted *= 1/(1-u)
        elif operand == (u-1)^2:
            extracted *= (1-u)
        elif operand == 1/(1+u)^2:
            extracted *= 1/(1+u)
        elif operand == 1/(u*x-1)^2:
            extracted *= 1/(1-u*x)
        else:
            new_radicand *= operand
    return extracted*sqrt(new_radicand)
def use_custom_sqrt(expression):
    return conditional_substitution(expression, pow(SR.wild(0), 1/2), custom_sqrt)

In [ ]:
assert use_custom_sqrt(identity_all).factor() == 0

In [ ]:
def G_factorial_moment_Narayana(d):
    return K((Delta * d.factorial()*alpha^d*NP(d-1, beta/alpha)).lift()).factor()

In [ ]:
G = L(SR(a*v), b*v^2)

In [ ]:
def G_factorial_moment(d):
    return G.diff(v, d).subs(v=1).factor().subs(branch_rules).factor()

In [ ]:
show(G_factorial_moment(1))

In [ ]:
assert all((G_factorial_moment(d)-G_factorial_moment_Narayana(d)).is_zero() for d in srange(2, 10))

In [ ]:
show(SR(G_factorial_moment_Narayana(2)))

In [ ]:
[factor(_) for _ in alpha]

In [ ]:
show([factor(_) for _ in beta/alpha])

In [ ]:
var('d, r')
SR_rd = SR.subring(accepting_variables=(r,d))
A.<Z> = AsymptoticRing('Z^QQ*log(Z)^QQ', SR_rd)
An.<n> = AsymptoticRing('QQ^n*n^QQ', SR_rd)
catalan_shifted_asy = asymptotic_expansions.Binomial_kn_over_n(n, k=2, precision=4).subs(n=n-1) / n
catalan_shifted_asy.show()

In [ ]:
def upsilon(z):
    return (1 - sqrt(1 - 4*z))/(2*z) - 1
z_asy = 1/4*(1-Z^-1)
u_asy = upsilon(z_asy)+O(Z^-3); u_asy

In [ ]:
x_asy = u_asy^r; x_asy

In [ ]:
def to_asy(expr):
    return fast_callable(SR(expr), vars=[u, x])(u_asy, x_asy)
def to_asy_with_delta(expr):
    return fast_callable(SR(expr.lift()), vars=[u, x, delta])(u_asy, x_asy, delta_asy)

In [ ]:
delta_asy = sqrt(to_asy(delta_square).map_coefficients(factor))

In [ ]:
Delta_asy = to_asy(Delta)
alpha_asy = to_asy_with_delta(alpha)
beta_asy = to_asy_with_delta(beta)

In [ ]:
def N0(d):
    return factorial(2*d-2)/(factorial(d)*factorial(d-1))
def N1(d):
    return factorial(2*d-2)/(factorial(d-1)*factorial(d-1))/2
def N2(d):
    return (d-2)*factorial(2*d-4)/(factorial(d-2)^2)
def NP_asy(d, expr):
    difference = expr - 1
    return N0(d) + N1(d)*difference + 1/2 * N2(d)*difference^2 + O(difference^3)


In [ ]:
factorial_moment_expansion = Delta_asy * SR_rd(d).factorial()*alpha_asy^d*NP_asy(d-1, beta_asy/alpha_asy)

In [ ]:
factorial_moment_asy = factorial_moment_expansion._singularity_analysis_(n, zeta=1/4, precision=2) / catalan_shifted_asy

In [ ]:
factorial_moment_asy = factorial_moment_asy.map_coefficients(lambda ex: ex.subs({
            gamma(d-1/2): factorial(2*d-2)/factorial(d-1) * sqrt(pi) * 2^(2-2*d),
            gamma(d-3/2): factorial(2*d-4)/factorial(d-2) * sqrt(pi) * 2^(4-2*d),
            gamma(d-1): factorial(d-2),
        }).subs({
            factorial(d): d*(d-1)*(d-2)*factorial(d-3),
            factorial(d-1): (d-1)*(d-2)*factorial(d-3),
            factorial(d-2): (d-2)*factorial(d-3),
            factorial(2*d-2): (2*d-2)*(2*d-3)*(2*d-4)*(2*d-5)*factorial(2*d-6),
            factorial(2*d-4): (2*d-4)*(2*d-5)*factorial(2*d-6),
        }).factor())

In [ ]:
show(factorial_moment_asy + O(n^(d-1)))

## Other Vertices

In [ ]:
alpha_plus_beta_prime = (2*a-4*a^2)/Delta^2
alpha_minus_beta_prime = 2*a/Delta
alpha_prime = ((alpha_plus_beta_prime + alpha_minus_beta_prime)/2).factor()
beta_prime = ((alpha_plus_beta_prime - alpha_minus_beta_prime)/2).factor()

In [ ]:
show(alpha_prime)

In [ ]:
show(beta_prime)

In [ ]:
identity_other = - L(a*(1+SR(q)), b) + Delta*(1/Delta -1 + (alpha_prime-beta_prime)*q)/2 \
    + Delta*T(alpha_prime*SR(q), beta_prime*q)

In [ ]:
assert use_custom_sqrt(identity_other).is_zero()